# Extracting a list of DOIs from `Elsapy`
---
In this notebook we describe methods to find and download papers to text mine their synthesis protocols. As this is a worked example, we go through the process with a single paper only.

The steps takes to find and download a paper are: 
1. Searching the SCOPUS database for a target paper using `elsapy`
2. Ientifying key metadata about the paper like the publisher and DOI
3. Using this metadata to download the paper


## Importing useful libraries

The first step is to import the `Elsapy` library and some ancillary tools.

In [1]:
from elsapy.elsclient import ElsClient #The elsevier input/output client
from elsapy.elsprofile import ElsAuthor, ElsAffil #Some useful information classes about the papers
from elsapy.elsdoc import FullDoc, AbsDoc # Different paper document types
from elsapy.elssearch import ElsSearch # The actual search engine we'll be using
import requests # For accessing REST APIs over the internet
import json # A common database system (javascript objects)
import numpy as np # Numerical python, essential for handling mathematical terms and matrices
import pandas as pd # A common data frame system, letting us access spreadsheet-likel data in python

## Setting up `Elsapy`
Then, using credentials from [dev.elsevier.com](https://dev.elsevier.com/), instantiate the `ElsClient` object. 

N.B. For this section, and institution token isn't needed, only the API key. However, if you have an institution token it must be used for the API key to work.

In [3]:
## Load configuration
conFile = open("./elsapy_config.json")
config = json.load(conFile)
conFile.close()

## Initialize client
client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']


## Defining search terms
Here, we instantiate an `ElsSearch` object with identical search terms as when we're searching the web of science. 
The filter syntax is somewhat complex, but we've showcased a few options here. 
Further instructions for search string entry can be found [here](https://dev.elsevier.com/sc_search_tips.html).

In [4]:
myDocSrch = ElsSearch('TITLE(ZIF-8 AND synthesis) AND SRCTYPE(J) AND pubyear > 2015 AND PUBLISHER(Elsevier)','scopus')

We separately run the search, and count how many results we get. In the first instance we set `get_all` to false to make it run faster and check we've found a reasonable number of papers.

In [5]:
myDocSrch.execute(client, get_all=False)
print(myDocSrch.tot_num_res, 'papers found')

98 papers found


Once done, we can re-run the search with `get_all` as True, from which we see the results in table form.

In [16]:
myDocSrch.execute(client, get_all=True)
myDocSrch.results_df[myDocSrch.results_df['openaccess']=='1']['pii']

2     S2352186423001657
13    S1387700322008668
30    S2590123022000482
37    S221171562200323X
41    S1387181121006132
69    S0277538718305412
70    S1878535218301606
Name: pii, dtype: object

The table contains lots of columns, many of which are not necessary for our purposes.

In [12]:
from pprint import pprint
print(len( myDocSrch.results_df.columns), 'columns')
pprint([x for x in myDocSrch.results_df.columns])

29 columns
['@_fa',
 'link',
 'prism:url',
 'dc:identifier',
 'eid',
 'dc:title',
 'dc:creator',
 'prism:publicationName',
 'prism:issn',
 'prism:volume',
 'prism:pageRange',
 'prism:coverDate',
 'prism:coverDisplayDate',
 'prism:doi',
 'pii',
 'citedby-count',
 'affiliation',
 'prism:aggregationType',
 'subtype',
 'subtypeDescription',
 'article-number',
 'source-id',
 'openaccess',
 'openaccessFlag',
 'prism:eIssn',
 'freetoread',
 'freetoreadLabel',
 'pubmed-id',
 'prism:issueIdentifier']


Instead, we can select just the columns we care about like this:

In [13]:
necessary_columns = ['dc:title', 'dc:creator', 'affiliation', 'prism:publicationName', 'prism:doi', 'pii']
paper_info = myDocSrch.results_df[necessary_columns]
paper_info.head(20)

,dc:title,dc:creator,affiliation,prism:publicationName,prism:doi,pii
0,Synthesis of nanofiltration membranes with enh...,Du Y.,"[{'@_fa': 'true', 'affilname': 'RMIT Universit...",Desalination,10.1016/j.desal.2023.116685,S001191642300317X
1,High catalytic performance of CoCuFe<inf>2</in...,Moghaddam F.M.,"[{'@_fa': 'true', 'affilname': 'Sharif Univers...",Journal of Molecular Structure,10.1016/j.molstruc.2023.135496,S0022286023005938
2,A high-performance ZIF-8 membrane for gas sepa...,Nguyen T.M.T.,"[{'@_fa': 'true', 'affilname': 'Chung Yuan Chr...",Environmental Technology and Innovation,10.1016/j.eti.2023.103169,S2352186423001657
3,Plasma-assisted synthesis of ZIF-8 membrane fo...,Shan Y.,"[{'@_fa': 'true', 'affilname': 'Jiangxi Normal...",Separation and Purification Technology,10.1016/j.seppur.2023.123871,S1383586623007797
4,Facile synthesis of ZIF-8-lignosulfonate micro...,Dai J.,"[{'@_fa': 'true', 'affilname': 'Qilu Universit...",International Journal of Biological Macromolec...,10.1016/j.ijbiomac.2023.124672,S0141813023015660
5,Synthesis and ciprofloxacin adsorption of Gum ...,Yang D.,"[{'@_fa': 'true', 'affilname': 'Wuhan Polytech...",Colloids and Surfaces A: Physicochemical and E...,10.1016/j.colsurfa.2023.131196,S0927775723002807
6,Molten NaCl assisted pyrolysis of ZIF-8/PAN el...,Ran S.,"[{'@_fa': 'true', 'affilname': 'Dalian Univers...",Chemical Engineering Journal,10.1016/j.cej.2023.142174,S1385894723009051
7,Bicomponent hydrogels assisted templating synt...,Zheng H.,"[{'@_fa': 'true', 'affilname': 'Henan Universi...",Journal of Molecular Structure,10.1016/j.molstruc.2022.134824,S002228602202470X
8,Facile synthesis of ZIF-8 incorporated electro...,Mohammed Y.A.Y.A.,"[{'@_fa': 'true', 'affilname': 'University of ...",Chemical Engineering Journal,10.1016/j.cej.2023.141972,S1385894723007039
9,An ultra-sensitive electrochemical aptasensor ...,Zhang Y.,"[{'@_fa': 'true', 'affilname': 'Chongqing Univ...",Microchemical Journal,10.1016/j.microc.2022.108316,S0026265X22011444


## Downloading the paper
Now that we've identified a paper that we want to download and textmine, we need to access it. 
While Elsevier allows easy access to their articles through the `elsapy` interface, I've found that accessing the web version simplifies later text minng. Accordingly, we're going to download the paper using `requests` alongside the paper identifier we found siogn `elsapy` and our acess tokens. 

This returns the manuscript as a binary xml document, which can be output to a file.

In [18]:
paper = paper_info.loc[30,'pii']

x = requests.get(
    f'https://api.elsevier.com/content/article/pii/{paper}',
    params = {
        'apiKey': config['apikey'],
        'insttoken': config['insttoken']
    }
)

with open(f'./{paper}.xml', 'wb') as f:
    f.write(x.content)